# 3.Eurpean Options Pricing with Black Scholes

In [7]:
import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt
import scipy.stats as st
#import statsmodels.datasets

from numpy.random import randn
from scipy.stats import norm
from scipy.optimize import fmin_slsqp
from scipy.optimize import fsolve
from scipy.interpolate import UnivariateSpline
from call_put_bs import call_put_bs
from min_max import min_max
from implied_vola import implied_vola
from mle_tlocation import mle_tlocation
from scipy.special import gamma
from yf import yf

According to Black-Scholes and Merton formula, the value of an european option with($\omega=1$ for a call, $\omega=-1$ for a put) is equal to:

\begin{equation*}
V(s,t;X,T,\sigma,r,q,\omega)=\omega\big(se^{-q(T-t)}N(\omega d_1)-Xe^{-r(T-t)}N(\omega d_2)\big)\;
\end{equation*}

\begin{equation*}
c=S_0N(d_1)-Ke^{-rT}N(d_2) 
\end{equation*}

\begin{equation*}
p=Ke^{-rT}N(-d_2) - S_0N(-d_1) 
\end{equation*}

where:

\begin{equation*}
d_1=\frac{ln({S_0 \over K}) + (r + {\sigma^2 \over 2})T}{\sigma{\sqrt T}}
\end{equation*}

and 

\begin{equation*}
d_2= \frac{ln({S_0 \over K}) + (r - {\sigma^2 \over 2})T}{\sigma{\sqrt T}} = d_1-\sigma{\sqrt T}
\end{equation*}

$T$=is the time to expiration <br/>
$K$=is the strike price <br/>
$\sigma$=volatility <br/>

**Example 1**

Contract parameters:$s_0=286.10$, $T=358/360$, $J=252$, $\sigma=0.0753$, $r=-0.0063$ and $q=0.0310$.

In [4]:
#S0=Underlying price
#q=Dividend
#J=Strike price
s0 = 286.1; T = 358/360; J = 252; sigma = 0.0753; r = -0.0063; q = 0.0310; # model- and contract parameters
min_max(s0,sigma,r,q,T,J,1,1,10**6)

33.69886900640171

**Example 2**

We calculate the implied volatility of the call option with strike $X = 76$ and market price $V^{\rm M} = 6.23$. Since there holds $s = 77.45$, $T − t = 99/360$, $r = -0.0081$ and $q = 0$ we find

In [8]:
s = 77.45; # the price of the underlying
T = yf((11,9,2019),[(20,12,2019)])[0]; r = -0.0081; q = 0; # time-to-maturity, cc risk-free
implied_vola(6.23,s,76,T,r,q,1,0.5)

0.3468833779533675

In [11]:
# load the downloaded data
df = pd.read_pickle("ublox")
df

,Strike price,Version number,Opening price,Daily high,Daily low,Last price,Daily settlem. price,Traded contracts,Open interest,Open interest (adj.)
0,0.01,0.0,0.0,0.0,0.0,0.0,77.44,0,0,0
1,48.00,0.0,0.0,0.0,0.0,0.0,29.57,0,0,0
2,50.00,0.0,0.0,0.0,0.0,0.0,27.62,0,0,0
3,52.00,0.0,0.0,0.0,0.0,0.0,25.67,0,0,0
4,56.00,0.0,0.0,0.0,0.0,0.0,21.88,0,0,0
5,60.00,0.0,0.0,0.0,0.0,0.0,18.21,0,0,0
6,64.00,0.0,0.0,0.0,0.0,0.0,14.73,0,0,0
7,68.00,0.0,0.0,0.0,0.0,0.0,11.51,0,4,4
8,72.00,0.0,0.0,0.0,0.0,0.0,8.66,0,0,0
9,76.00,0.0,0.0,0.0,0.0,0.0,6.23,0,0,0


In [12]:
df = df[['Strike price','Daily settlem. price']].astype(float)
D = df.values
D = D[D[:,0]>=1] # do not take all values to fit the model
X = D[:,0]; VM = D[:,1]

Find the implied Volatility

In [13]:
vola_i = np.zeros(len(D))
vola_i[0] = implied_vola(D[0,1],s,D[0,0],T,r,q,1,0.6);
for j in range(1,len(D)):
    vola_i[j] = implied_vola(VM[j],s,X[j],T,r,q,1,vola_i[j-1]);

vola_i

array([0.50370862, 0.48556632, 0.46447264, 0.43892689, 0.41479637,
       0.39377892, 0.37492644, 0.36004817, 0.34688338, 0.33817546,
       0.33619289, 0.33769768, 0.34224473, 0.34779472])